In [ ]:
%load_ext autoreload
%autoreload 2

import uproot
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cbook as cbook
import numpy as np
import pandas as pd
from decimal import Decimal
from scipy.stats import norm
from scipy.optimize import curve_fit
from scipy import stats, interpolate
import scipy
import warnings
import datetime as dt
import landau

import importlib

# local imports
from lib.constants import *

from scipy.integrate import quad, tplquad
from scipy.special import erf
from scipy.interpolate import CubicSpline

In [ ]:
dosave = False
savedir = "./"
plt.rcParams.update({'font.size': 14})
plotqual = "_mc"
plottitle = "MC"
tpcnames = ["EE", "EW", "WE", "WW"]

In [ ]:
perhit_df = pd.read_hdf("data.df")

In [ ]:
perhit_df

In [ ]:
NCHUNK = 10

In [ ]:
perhit_df["tpcEE"] = perhit_df.tpcE & (perhit_df.cryostat == 0)
perhit_df["tpcEW"] = perhit_df.tpcW & (perhit_df.cryostat == 0)
perhit_df["tpcWE"] = perhit_df.tpcE & (perhit_df.cryostat == 1)
perhit_df["tpcWW"] = perhit_df.tpcW & (perhit_df.cryostat == 1)

perhit_df["dqdx"] = perhit_df.charge / perhit_df.true_pitch
perhit_df["true_dedx"] = perhit_df.true_e / perhit_df.true_pitch
perhit_df["thit"] = (perhit_df.time * tick_period - perhit_df.pandora_t0 - tanode*tick_period) / 1000.


In [ ]:
def landau_gaus(X, *p):
    mpv, eta, sigma, A = p
    sigma = np.minimum(sigma, 100*eta)
    return landau.landau.gauss_landau(X, mpv, eta, sigma, A)


In [ ]:
tbins = np.linspace(0, 1000, 21)
tbins = tbins[:-1]
tcenters = (tbins[1:] + tbins[:-1])/2.

qbins = np.linspace(100, 4000, 39*4+1)
qbin_centers = (qbins[1:] + qbins[:-1]) / 2.

when = (perhit_df.pitch < 0.4*NCHUNK)
tpcs = [perhit_df.tpcEE, perhit_df.tpcEW, perhit_df.tpcWE, perhit_df.tpcWW]

In [ ]:
Means = {}
MPVs = {}
MPV_errs = {}
locs = {}
loc_errs = {}

etas = {}
sigmas = {}

for tpc in tpcnames:
    Means[tpc] = []
    MPVs[tpc] = []
    MPV_errs[tpc] = []
    locs[tpc] = []
    loc_errs[tpc] = []
    etas[tpc] = []
    sigmas[tpc] = []
    

iplt = 0
for tpc, tname in zip(tpcs, tpcnames):
    for i, (blo, bhi) in enumerate(zip(tbins[:-1], tbins[1:])):
        plt.figure(iplt)
        thiswhen = tpc & when & (perhit_df.thit > blo) & (perhit_df.thit < bhi)

        N,_,_ = plt.hist(perhit_df.dqdx[thiswhen], bins=qbins)

        maxbin = np.argmax(N)
        when_fit = np.abs(np.array(range(len(qbin_centers))) - maxbin) < 15 # within 15 bins

        mpv_lo = 500
        mpv_hi = 1000
        mpv0 = 750
        p0 = [mpv0, 50, 50, np.max(N)*400.]
        bounds = ([mpv_lo, 10, 10, np.max(N)*400./100.], [mpv_hi, 250, 250, np.max(N)*400.*100.])
        popt, perr = curve_fit(landau_gaus, qbin_centers[when_fit], N[when_fit], 
                               p0=p0, maxfev=10_000, sigma=np.maximum(np.sqrt(N),1)[when_fit], bounds=bounds)

        plt.plot(qbin_centers, landau_gaus(qbin_centers, *popt), 
                 label="Fit")
        mpv = popt[0] - 0.228*popt[1]

        plt.text(0.45, 0.175, "%i Wire\n%.0f < time < %.0f $\\mu$s\n\nMPV = %.2f ADDC/cm\n$\\mu$ = %.2f ADDC/cm\n\n$\eta$ = %.2f ADDC/cm\n$\sigma$ = %.2f ADDC/cm" %
                 tuple([NCHUNK, blo, bhi, mpv] + list(popt[:3])),
                 fontsize=14, transform=plt.gca().transAxes)
        plt.xlim([0, 3000])
        plt.xlabel("dQ/dx [ADDC/cm]")
        plt.ylabel("Entries")
        plt.title(plottitle + " TPC " + tname)
        plt.tight_layout()

        if dosave:
            plt.savefig(savedir + "%iwire_t_%i_%i_dqdx_TPC%s%s.png" % (NCHUNK, int(blo), int(bhi), tname, plotqual))

        Means[tname].append(np.mean(stats.trim1(perhit_df.dqdx[thiswhen], 0.2)))
        MPVs[tname].append(popt[0] - 0.228*popt[1])
        MPV_errs[tname].append(np.sqrt(np.diag(perr)[0] + (0.228**2)*np.diag(perr)[1] - 0.228*(perr[0,1] + perr[1,0])))

        locs[tname].append(popt[0])
        loc_errs[tname].append(np.sqrt(np.diag(perr))[0])

        etas[tname].append(popt[1])
        sigmas[tname].append(popt[2])
        
        iplt += 1
    
for tpc in tpcnames:
    Means[tpc] = np.array(Means[tpc])
    MPVs[tpc] = np.array(MPVs[tpc])
    MPV_errs[tpc] = np.array(MPV_errs[tpc])
    locs[tpc] = np.array(locs[tpc])
    loc_errs[tpc] = np.array(loc_errs[tpc])
    etas[tpc] = np.array(etas[tpc])
    sigmas[tpc] = np.array(sigmas[tpc])

In [ ]:
whent = (tcenters > 100) & (tcenters < 900)

In [ ]:
def lifetime(t, Q0, tau):
    return Q0*np.exp(-t/tau)

p0 = [500, 3e3]
bounds = ([0.0, 0.0], [1500., 10e3])

In [ ]:
for i,tpc in enumerate(tpcnames):
    plt.figure(i)
    plt.errorbar(tcenters[whent], MPVs[tpc][whent], yerr=MPV_errs[tpc][whent], label="Data")
    
    popt, perr = curve_fit(lifetime, tcenters[whent], MPVs[tpc][whent], 
                       p0=p0, maxfev=10_000, sigma=MPV_errs[tpc][whent], bounds=bounds)
    plt.plot(tcenters[whent], lifetime(tcenters[whent], *popt), label="Fit")
    plt.text(0.05, 0.05, "$\\tau = %.0f \\pm %.0f \\mu$s" % (popt[1], np.sqrt(np.diag(perr))[1]),
            fontsize=16, transform=plt.gca().transAxes)
    
    plt.xlabel("Hit Time [$\\mu$s]")
    plt.ylabel("MPV dE/dx [MeV/cm]")
    plt.title(plottitle + " TPC " + tpc + " %i-Wire" % NCHUNK)
    plt.tight_layout()
    if dosave: plt.savefig(savedir + "%iwire_MPVdqdx_time_TPC%s%s.png" % (NCHUNK, tpc, plotqual))

In [ ]:
for i, tpc in enumerate(tpcnames):
    plt.figure(i)
    plt.plot(tcenters[whent], Means[tpc][whent])
    plt.xlabel("Hit Time [$\\mu$s]")
    plt.ylabel("Mean dQ/dx [ADDC/cm]")
    plt.title(plottitle + " TPC " + tpc + " %i-Wire" % NCHUNK)
    plt.tight_layout()
    if dosave: plt.savefig(savedir + "%iwire_Meandqdx_time_TPC%s%s.png" % (NCHUNK, tpc, plotqual))

In [ ]:
for i, tpc in enumerate(tpcnames):
    plt.figure(i)
    plt.plot(tcenters[whent], sigmas[tpc][whent])
    plt.xlabel("Hit Time [$\\mu$s]")
    plt.ylabel("Fit $\\sigma$ [ADDC/cm]")
    plt.title(plottitle + " TPC " + tpc + " %i-Wire" % NCHUNK)
    plt.tight_layout()
    if dosave: plt.savefig(savedir + "%iwire_sigma_time_TPC%s%s.png" % (NCHUNK, tpc, plotqual))

In [ ]:
for i, tpc in enumerate(tpcnames):
    plt.figure(i)
    plt.plot(tcenters[whent], etas[tpc][whent])
    plt.xlabel("Hit Time [$\\mu$s]")
    plt.ylabel("Fit $\\eta$ [ADDC/cm]")
    plt.title(plottitle + " TPC " + tpc + " %i-Wire" % NCHUNK)
    plt.tight_layout()
    if dosave: plt.savefig(savedir + "%iwire_eta_time_TPC%s%s.png" % (NCHUNK, tpc, plotqual))